In [64]:
import os
%matplotlib inline

import os
from collections import Counter, OrderedDict
import numpy as np
from operator import itemgetter
import matplotlib.pyplot as plt
from astropy.table import Table
import multiprocessing
from cesium.time_series import TimeSeries
import cesium.featurize as featurize
from tqdm import tnrange, tqdm_notebook, tqdm
import sklearn 
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
import seaborn as sns
import pandas as pd

In [65]:
### Read data files train_meta and train
os.chdir('/Users/shengyao/Downloads/Kaggle Data')

In [66]:
train_meta_data = pd.read_csv('training_set_metadata.csv')
train = pd.read_csv('training_set.csv')
train['mjd'] = train['mjd'].apply(int)

In [67]:
train.head()

,object_id,mjd,passband,flux,flux_err,detected
0,615,59750,2,-544.810303,3.622952,1
1,615,59750,1,-816.434326,5.553370,1
2,615,59750,3,-471.385529,3.801213,1
3,615,59750,4,-388.984985,11.395031,1
4,615,59752,2,-681.858887,4.041204,1


In [68]:
part1 = train[['object_id','mjd','passband','flux']]
part2 = train[['object_id','mjd','passband','flux_err']]
part3 = train[['object_id','mjd','passband','detected']]

In [69]:
part1 = pd.melt(part1,id_vars=['object_id','mjd','passband'],value_name='flux').pivot_table(index =['object_id','mjd'],columns='passband')
part2 = pd.melt(part2,id_vars=['object_id','mjd','passband'],value_name='flux_err').pivot_table(index =['object_id','mjd'],columns='passband')
part3 = pd.melt(part3,id_vars=['object_id','mjd','passband'],value_name='detected').pivot_table(index =['object_id','mjd'],columns='passband')

In [70]:
output = (part1.join(part2)).join(part3)

In [71]:
output

flux                                                   \
passband                  0            1           2           3           4   
object_id mjd                                                                  
615       59750         NaN  -816.434326 -544.810303 -471.385529 -388.984985   
          59752         NaN -1061.457031 -681.858887 -524.954590 -393.480225   
          59767         NaN  -815.188599 -548.013550 -475.516052 -405.663818   
          59770         NaN  -820.042786 -554.903198 -477.004730 -400.270386   
          59779         NaN  -921.002502 -630.523682 -518.533997 -422.184509   
          59782         NaN  -449.095612 -280.039520 -316.704865 -332.885437   
          59797         NaN    35.511822  391.399231  330.623901  360.397858   
          59800         NaN   129.541901  168.739899   30.120724  -60.942333   
          59807         NaN  -420.796417 -256.660980 -298.936859 -311.977783   
          59810         NaN  -527.020325 -342.819763 -363.282532 -348.628662   
          59813         NaN -1100.440063 -678.045715 -506.687408 -304.049713   
          59819    6.878784          NaN         NaN         NaN         NaN   
          59820   39.364853          NaN         NaN         NaN         NaN   
          59821  -10.422381          NaN         NaN         NaN         NaN   
          59822  -65.485130          NaN         NaN         NaN         NaN   
          59823 -113.349159          NaN         NaN         NaN         NaN   
          59835         NaN  -178.149399  -54.949490 -140.818436 -200.294128   
          59839         NaN  -953.883728 -639.035950 -518.293274 -418.723907   
          59842         NaN -1003.971497 -502.215332 -233.167755  111.507675   
          59851  -68.502457          NaN         NaN         NaN         NaN   
          59854         NaN   217.894211  459.452667  361.023438  374.446442   
          59857         NaN   646.523193  599.812195  354.961365  293.879608   
          59864         NaN  -942.167908 -637.105347 -524.586548 -414.447723   
          59867         NaN  -910.677734 -332.763123  -62.065010  202.288223   
          59870         NaN   659.486694  604.344543  373.986511  322.604034   
          59873         NaN   -98.796974    4.656033  -93.732880 -165.793457   
          59874  -97.353195          NaN         NaN         NaN         NaN   
          59875  -97.523880          NaN         NaN         NaN         NaN   
          59876 -108.672577          NaN         NaN         NaN         NaN   
          59877 -116.913223          NaN         NaN         NaN         NaN   
...                     ...          ...         ...         ...         ...   
130779836 60464         NaN          NaN    3.294472         NaN         NaN   
          60466  113.988297          NaN         NaN         NaN         NaN   
          60490         NaN          NaN   12.464979         NaN         NaN   
          60491         NaN   -13.734149         NaN         NaN         NaN   
          60496         NaN          NaN  -14.478923         NaN         NaN   
          60497         NaN          NaN         NaN   12.178056         NaN   
          60498         NaN          NaN         NaN  -20.815857         NaN   
          60499   54.361107          NaN         NaN         NaN         NaN   
          60503         NaN          NaN         NaN    9.912425         NaN   
          60505         NaN          NaN   17.719660         NaN         NaN   
          60506         NaN          NaN         NaN         NaN         NaN   
          60510         NaN          NaN         NaN         NaN  -39.215195   
          60515         NaN          NaN         NaN         NaN         NaN   
          60524  136.082718          NaN         NaN         NaN         NaN   
          60525   43.910740          NaN         NaN         NaN         NaN   
          60526         NaN          NaN         NaN   59.376686         NaN   
          60528        

In [103]:
df = pd.merge(output.reset_index(),train_meta_data,on='object_id',copy=False)

/anaconda3/lib/python3.6/site-packages/pandas/core/reshape/merge.py:558: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


In [104]:
df

,object_id,"(object_id, )","(mjd, )","(flux, 0)","(flux, 1)","(flux, 2)","(flux, 3)","(flux, 4)","(flux, 5)","(flux_err, 0)",...,decl,gal_l,gal_b,ddf,hostgal_specz,hostgal_photoz,hostgal_photoz_err,distmod,mwebv,target
0,615,615,59750,NaN,-816.434326,-544.810303,-471.385529,-388.984985,NaN,NaN,...,-61.943836,320.796530,-51.753706,1,0.0,0.0,0.0,NaN,0.017,92
1,615,615,59752,NaN,-1061.457031,-681.858887,-524.954590,-393.480225,-355.886780,NaN,...,-61.943836,320.796530,-51.753706,1,0.0,0.0,0.0,NaN,0.017,92
2,615,615,59767,NaN,-815.188599,-548.013550,-475.516052,-405.663818,-421.199066,NaN,...,-61.943836,320.796530,-51.753706,1,0.0,0.0,0.0,NaN,0.017,92
3,615,615,59770,NaN,-820.042786,-554.903198,-477.004730,-400.270386,-415.286896,NaN,...,-61.943836,320.796530,-51.753706,1,0.0,0.0,0.0,NaN,0.017,92
4,615,615,59779,NaN,-921.002502,-630.523682,-518.533997,-422.184509,-422.815094,NaN,...,-61.943836,320.796530,-51.753706,1,0.0,0.0,0.0,NaN,0.017,92
5,615,615,59782,NaN,-449.095612,-280.039520,-316.704865,-332.885437,-365.075775,NaN,...,-61.943836,320.796530,-51.753706,1,0.0,0.0,0.0,NaN,0.017,92
6,615,615,59797,NaN,35.511822,391.399231,330.623901,360.397858,369.439667,NaN,...,-61.943836,320.796530,-51.753706,1,0.0,0.0,0.0,NaN,0.017,92
7,615,615,59800,NaN,129.541901,168.739899,30.120724,-60.942333,-128.920334,NaN,...,-61.943836,320.796530,-51.753706,1,0.0,0.0,0.0,NaN,0.017,92
8,615,615,59807,NaN,-420.796417,-256.660980,-298.936859,-311.977783,-344.536072,NaN,...,-61.943836,320.796530,-51.753706,1,0.0,0.0,0.0,NaN,0.017,92
9,615,615,59810,NaN,-527.020325,-342.819763,-363.282532,-348.628662,-391.271271,NaN,...,-61.943836,320.796530,-51.753706,1,0.0,0.0,0.0,NaN,0.017,92
